# RFNSA Antenna Details
##### ©Haris Hassan


In [1]:
from IPython import get_ipython;   
get_ipython().magic('reset -sf')
import pandas as pd
pd.set_option('display.max_colwidth', None)
from IPython.display import display, HTML
import re
import math
from itertools import count
class text_format:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'
data = pd.read_excel(r'C:\Users\Mewtwo\Downloads\OSBORNE PARK WA 6017.xlsx')
# Function to convert from mW to dBm
def mW2dBm(mW):
    return 10.*log10(mW)

# Function to convert from dBm to mW
def dBm2mW(dBm):
    return 10**((dBm)/10.)



df = pd.DataFrame(data, columns=['Antenna ID No','Antenna Model','Sector','Height - Phase Centre (m)'
                                 ,'Bearing Degrees (true)',
                                 'Mech Downtilt','Elect Downtilt','System',
                                 'Port Number (Band Power per Port (dBm))','Band Power per Port (dBm)'])
df = df.rename(columns={'Antenna Model': 'Antenna', 'Antenna ID No': 'ID', 'Bearing Degrees (true)': 'Bearing'
                  , 'Mech Downtilt': 'MDT', 'Elect Downtilt': 'EDT', 'Height - Phase Centre (m)': 'Height', 
                   'Port Number (Band Power per Port (dBm))': 'Possible Ports', 
                   'Band Power per Port (dBm)': 'Powers'})
df = df.drop(df[df['Fee'] >= 24000].index, inplace = True)



#Extracting Carrier (optus, telstra, Vodafone), technology (LTE, NR, WCDMA) and Frequency information from System 
df['System'] = df['System'].str.replace(' \[', ';', regex=True)
df['System'] = df['System'].str.replace('/', ';', 1, regex=True)
df['Carrier'] = df['System'].str.split(" ;").str[0]

df['Tech'] = df['System'].str.split(";").str[1]
#display(df['Carrier'])

df.sort_values(by=['Carrier','ID'], ascending=[True, True], inplace = True)
df['Sector'] = df['Sector'].astype(str).replace('\.0', '', regex=True)

#Reset index of dataframe after sorting
df = df.reset_index(drop=True)
df.index += 1

#display( HTML( df.to_html().replace("\\n","<br><li type='1'>") ) )

df['EDT'] = df['EDT'].str.replace(' to ', '-', regex=True)
df['EDT'] = df['EDT'].str.replace(r'\(.*\)','', regex=True)
df['Powers'] = df['Powers'].str.replace('49', '79.4 W')
df['Powers'] = df['Powers'].str.replace('48', '63.1 W')
df['Powers'] = df['Powers'].str.replace('47', '50.1 W')
df['Powers'] = df['Powers'].str.replace('46', '39.8 W')
df['Powers'] = df['Powers'].str.replace('45', '31.6 W')
df['Powers'] = df['Powers'].str.replace('44', '25.1 W')
df['Powers'] = df['Powers'].str.replace('43', '20 W')
df['Powers'] = df['Powers'].str.replace('42', '15.8 W')
df['Powers'] = df['Powers'].str.replace('37', '5 W')
df['Powers'] = df['Powers'].str.replace('18.91', '0.0778 W')

df['Sector']= [''.join(''.join(map(str, m))+' - Sector ' +f'{l}') for l, m in zip(df['Sector'],df['Tech'])]

df.fillna(value = 0,inplace = True)

C:\Users\Mewtwo\anaconda3\lib\site-packages\ipykernel_launcher.py:68: FutureWarning: The default value of regex will change from True to False in a future version.


In [2]:
#### what should be the frequency
Assess_Freq_list = []
for i in df['Tech']:
    if any([x in i for x in ['GSM900', 'WCDMA850', 'WCDMA900','LTE850', 'NR850','LTE900']]):
        Assess_Freq_list.append('900')
    elif 'LTE700' in i:
        Assess_Freq_list.append('750')
    elif 'LTE2300' in i:
        Assess_Freq_list.append('2350')
    else:
        Assess_Freq_list.append(i)
df['Assess Freq'] = Assess_Freq_list

In [3]:
####Where to add powers?########
Mylist =[]

for i in df['Powers']:
    count = 1
    lister = []
    if isinstance(i, str):
        for j in i:
            if j==';':
                count+=1
            else:
                lister.append(count)
        Mylist.append(lister)
    else:
        Mylist.append(list(str(i))) 
        
    
#display(Mylist)
new_k = []
for elem in Mylist:
    kiterator=[]
    for el in elem:
        if el not in kiterator:
            kiterator.append(el)
    new_k.append(kiterator)
k = new_k

df['WhereToAddPower'] = k
#display(k)

In [4]:
###############Wat should the power be?######
df['Powerss'] = df['Powers'].str.split(";")

itArr2 = []
for index, elm in df['Powerss'].iteritems():
    if isinstance(elm, (list, tuple)):
        elm = [sub.replace('W', '') for sub in elm]
        itArr2.append([x for x in elm if x != ''])
    else:
        itArr2.append([elm])
        
df['UnformatedPowers'] = itArr2


#display(df['UnformatedPowers'])
df['Power (W)'] = ['+'.join(map(str, l)) for l in df['UnformatedPowers']]



In [5]:
####Formatting Powers Column with ports numbers

#itArr=[]
#for index, elm in df['Powers'].iteritems():
#    if isinstance(elm, (list, tuple)):
#        itArr.append([f'{i+1}. {l}' for i, l in enumerate(elm)])
#    else:
#        itArr.append([elm])
#df=df.drop(columns='Powers', axis=1)
#df.insert(2, 'Powers', itArr)
#############################################

## Add these Antennas to prox5

In [6]:
AntennasUnique = list( dict.fromkeys(df['Antenna']) )
print('You need to add these antennas\n\n'+str.join(" \n", AntennasUnique)+' \n\nto PROX5')

You need to add these antennas

ASI4517R1 
AEQP 
AEQN 
AENB 
RRV4S4-65D-R5-V4 [Service Beam] 
RR2VV-6533D-R6 
AEQE 

to PROX5


In [7]:
IDUnique = list( dict.fromkeys(df['ID'],df['Antenna']) )

## IDs for Antennas

In [8]:
AntennaIds = {}
for antenna in AntennasUnique:
    IdsForAntenna = list(dict.fromkeys(df.loc[df['Antenna'] == antenna, 'ID']))
    AntennaIds[antenna] = IdsForAntenna

for x, y in AntennaIds.items():
    print(f'\nThe id of '+text_format.BOLD+ x + text_format.END + ' are\n'+str.join(" \n", y))


The id of ASI4517R1 are
11-O 
21-O 
31-O

The id of AEQP are
12-O 
22-O 
32-O

The id of AEQN are
12-O 
22-O 
32-O

The id of AENB are
13-O 
23-O 
33-O

The id of RRV4S4-65D-R5-V4 [Service Beam] are
A1 
A2 
A3

The id of RR2VV-6533D-R6 are
11-V 
21-V 
31-V

The id of AEQE are
12-V 
22-V 
32-V


## Antenna Settings

In [9]:
display('Set Power and Frequency to ports in Prox5')
Settings = {}
for i in IDUnique:
    IdsForAntenna = list(dict.fromkeys(df.loc[df['ID'] == i, 'ID']))
    AntennaIds[antenna] = IdsForAntenna

for i in IDUnique:
        print(f'\nFor '+text_format.BOLD+i+text_format.END+f', the Settings are')
        display(df.loc[(df['ID'] == i),['ID','Height','Bearing','MDT']])

'Set Power and Frequency to ports in Prox5'


For 11-O, the Settings are


,ID,Height,Bearing,MDT
1,11-O,21.55,0,0
2,11-O,21.55,0,0
3,11-O,21.55,0,0
4,11-O,21.55,0,0
5,11-O,21.55,0,0
6,11-O,21.55,0,0
7,11-O,21.55,0,0
8,11-O,21.55,0,0
9,11-O,21.55,0,0
10,11-O,21.55,0,0



For 12-O, the Settings are


,ID,Height,Bearing,MDT
16,12-O,22.35,0,0
17,12-O,22.35,0,0


...

,ID,Height,Bearing,MDT
92,22-V,25.24,170,0



For 31-V, the Settings are


,ID,Height,Bearing,MDT
93,31-V,24.59,295,0
94,31-V,24.59,295,0
95,31-V,24.59,295,0
96,31-V,24.59,295,0
97,31-V,24.59,295,0
98,31-V,24.59,295,0



For 32-V, the Settings are


,ID,Height,Bearing,MDT
99,32-V,25.24,295,2


## Set Power and Frequency to ports in Prox5

In [10]:
display('Set Power and Frequency to ports in Prox5')
for i in IDUnique:
        print(text_format.BOLD+i)
        display(df.loc[(df['ID'] == i),['ID','Tech','Possible Ports','WhereToAddPower','Power (W)']])

'Set Power and Frequency to ports in Prox5'

11-O


,ID,Tech,Possible Ports,WhereToAddPower,Power (W)
1,11-O,WCDMA900,1;2;3;4,[3],63.1
2,11-O,WCDMA2100,5;6;7;8;9;10;11;12,[5],31.6
3,11-O,LTE2600,5;6;7;8;9;10;11;12,"[5, 6, 7, 8]",50.1 +50.1 +50.1 +50.1
4,11-O,NR/LTE2100,5;6;7;8;9;10;11;12,"[9, 10, 11, 12]",50.1 +50.1 +50.1 +50.1
5,11-O,LTE900,1;2;3;4,"[1, 2]",63.1 +63.1
6,11-O,LTE700,1;2;3;4,"[1, 2, 3, 4]",63.1 +63.1 +63.1 +63.1
7,11-O,LTE2300,5;6;7;8;9;10;11;12,"[5, 6, 7, 8]",31.6 +31.6 +31.6 +31.6
8,11-O,LTE1800,5;6;7;8;9;10;11;12,"[9, 10, 11, 12]",50.1 +50.1 +50.1 +50.1
9,11-O,LTE2100,5;6;7;8;9;10;11;12,"[5, 6]",31.6 +31.6
10,11-O,LTE1800,5;6;7;8;9;10;11;12,"[5, 6, 7, 8]",50.1 +50.1 +50.1 +50.1


12-O


,ID,Tech,Possible Ports,WhereToAddPower,Power (W)
16,12-O,NR3500,1;2,[1],79.4
17,12-O,NR3500,1;2,[1],50.1


...

,ID,Tech,Possible Ports,WhereToAddPower,Power (W)
92,22-V,NR3500,1;2,[1],50.1


31-V


,ID,Tech,Possible Ports,WhereToAddPower,Power (W)
93,31-V,NR/LTE700,1;2;3;4,"[1, 2, 3, 4]",50.1 +50.1 +50.1 +50.1
94,31-V,NR/LTE850,1;2;3;4,"[1, 2, 3, 4]",50.1 +50.1 +50.1 +50.1
95,31-V,NB-IOT900,1;2;3;4,"[1, 4]",15.8 +15.8
96,31-V,WCDMA900,1;2;3;4,"[1, 4]",50.1 +50.1
97,31-V,NR/LTE1800,5;6;7;8;9;10;11;12,"[5, 6, 7, 8, 9, 10, 11, 12]",50.1 +50.1 +50.1 +50.1 +50.1 +50.1 +50.1 +50.1
98,31-V,NR/LTE2100,5;6;7;8;9;10;11;12,"[5, 6, 7, 8, 9, 10, 11, 12]",50.1 +50.1 +50.1 +50.1 +50.1 +50.1 +50.1 +50.1


32-V


,ID,Tech,Possible Ports,WhereToAddPower,Power (W)
99,32-V,NR3500,1;2,[1],50.1


## EMEG Equipment List

In [11]:
display('EMEG Equipment List')
display(df[['ID','EDT','Sector','WhereToAddPower','Power (W)']])

'EMEG Equipment List'

,ID,EDT,Sector,WhereToAddPower,Power (W)
1,11-O,0-12,WCDMA900 - Sector 1,[3],63.1
2,11-O,0-12,WCDMA2100 - Sector 1,[5],31.6
3,11-O,0-12,LTE2600 - Sector 1,"[5, 6, 7, 8]",50.1 +50.1 +50.1 +50.1
4,11-O,0-12,NR/LTE2100 - Sector 1,"[9, 10, 11, 12]",50.1 +50.1 +50.1 +50.1
5,11-O,0-12,LTE900 - Sector 1,"[1, 2]",63.1 +63.1
...,...,...,...,...,...
95,31-V,2-12,NB-IOT900 - Sector 3,"[1, 4]",15.8 +15.8
96,31-V,2-12,WCDMA900 - Sector 3,"[1, 4]",50.1 +50.1
97,31-V,2-12,NR/LTE1800 - Sector 3,"[5, 6, 7, 8, 9, 10, 11, 12]",50.1 +50.1 +50.1 +50.1 +50.1 +50.1 +50.1 +50.1
98,31-V,2-12,NR/LTE2100 - Sector 3,"[5, 6, 7, 8, 9, 10, 11, 12]",50.1 +50.1 +50.1 +50.1 +50.1 +50.1 +50.1 +50.1


## Exports

In [12]:
display(df)

,ID,Antenna,Sector,Height,Bearing,MDT,EDT,System,Possible Ports,Powers,Carrier,Tech,Assess Freq,WhereToAddPower,Powerss,UnformatedPowers,Power (W)
1,11-O,ASI4517R1,WCDMA900 - Sector 1,21.55,0,0,0-12,Optus ; WCDMA900;Macro],1;2;3;4,;;63.1 W;;;;;;;;;,Optus,WCDMA900,900,[3],"[, , 63.1 W, , , , , , , , , ]",[63.1 ],63.1
2,11-O,ASI4517R1,WCDMA2100 - Sector 1,21.55,0,0,0-12,Optus ; WCDMA2100;Macro],5;6;7;8;9;10;11;12,;;;;31.6 W;;;;;;;,Optus,WCDMA2100,WCDMA2100,[5],"[, , , , 31.6 W, , , , , , , ]",[31.6 ],31.6
3,11-O,ASI4517R1,LTE2600 - Sector 1,21.55,0,0,0-12,Optus ; LTE2600;Macro],5;6;7;8;9;10;11;12,;;;;50.1 W;50.1 W;50.1 W;50.1 W;;;;,Optus,LTE2600,LTE2600,"[5, 6, 7, 8]","[, , , , 50.1 W, 50.1 W, 50.1 W, 50.1 W, , , , ]","[50.1 , 50.1 , 50.1 , 50.1 ]",50.1 +50.1 +50.1 +50.1
4,11-O,ASI4517R1,NR/LTE2100 - Sector 1,21.55,0,0,0-12,Optus ; NR/LTE2100;Macro],5;6;7;8;9;10;11;12,;;;;;;;;50.1 W;50.1 W;50.1 W;50.1 W,Optus,NR/LTE2100,NR/LTE2100,"[9, 10, 11, 12]","[, , , , , , , , 50.1 W, 50.1 W, 50.1 W, 50.1 W]","[50.1 , 50.1 , 50.1 , 50.1 ]",50.1 +50.1 +50.1 +50.1
5,11-O,ASI4517R1,LTE900 - Sector 1,21.55,0,0,0-12,Optus ; LTE900;Macro],1;2;3;4,63.1 W;63.1 W;;;;;;;;;;,Optus,LTE900,900,"[1, 2]","[63.1 W, 63.1 W, , , , , , , , , , ]","[63.1 , 63.1 ]",63.1 +63.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,31-V,RR2VV-6533D-R6,NB-IOT900 - Sector 3,24.59,295,0,2-12,Vodafone ; NB-IOT900;Macro],1;2;3;4,15.8 W;;;15.8 W;;;;;;;;,Vodafone,NB-IOT900,NB-IOT900,"[1, 4]","[15.8 W, , , 15.8 W, , , , , , , , ]","[15.8 , 15.8 ]",15.8 +15.8
96,31-V,RR2VV-6533D-R6,WCDMA900 - Sector 3,24.59,295,0,2-12,Vodafone ; WCDMA900;Macro],1;2;3;4,50.1 W;;;50.1 W;;;;;;;;,Vodafone,WCDMA900,900,"[1, 4]","[50.1 W, , , 50.1 W, , , , , , , , ]","[50.1 , 50.1 ]",50.1 +50.1
97,31-V,RR2VV-6533D-R6,NR/LTE1800 - Sector 3,24.59,295,0,2-12,Vodafone ; NR/LTE1800;Macro],5;6;7;8;9;10;11;12,;;;;50.1 W;50.1 W;50.1 W;50.1 W;50.1 W;50.1 W;50.1 W;50.1 W,Vodafone,NR/LTE1800,NR/LTE1800,"[5, 6, 7, 8, 9, 10, 11, 12]","[, , , , 50.1 W, 50.1 W, 50.1 W, 50.1 W, 50.1 W, 50.1 W, 50.1 W, 50.1 W]","[50.1 , 50.1 , 50.1 , 50.1 , 50.1 , 50.1 , 50.1 , 50.1 ]",50.1 +50.1 +50.1 +50.1 +50.1 +50.1 +50.1 +50.1
98,31-V,RR2VV-6533D-R6,NR/LTE2100 - Sector 3,24.59,295,0,2-12,Vodafone ; NR/LTE2100;Macro],5;6;7;8;9;10;11;12,;;;;50.1 W;50.1 W;50.1 W;50.1 W;50.1 W;50.1 W;50.1 W;50.1 W,Vodafone,NR/LTE2100,NR/LTE2100,"[5, 6, 7, 8, 9, 10, 11, 12]","[, , , , 50.1 W, 50.1 W, 50.1 W, 50.1 W, 50.1 W, 50.1 W, 50.1 W, 50.1 W]","[50.1 , 50.1 , 50.1 , 50.1 , 50.1 , 50.1 , 50.1 , 50.1 ]",50.1 +50.1 +50.1 +50.1 +50.1 +50.1 +50.1 +50.1


In [13]:
#df3 = pd.DataFrame.from_dict(AntennaIds)
df3 = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in AntennaIds.items() ]))
df3 = df3.fillna('')

dfAntennaSettings = df[['ID','Height','Bearing','MDT']].copy()
dfAntennaSettings=dfAntennaSettings.drop_duplicates(keep='first')

path = r'C:\Users\Mewtwo\Desktop'+'\\'+ data['Site Name'][3] + '.xlsx'
#writer = pd.ExcelWriter(path, engine = 'xlsxwriter')
#writer.column_dimensions[column].bestFit = True



from openpyxl.utils import get_column_letter
from openpyxl import Workbook
from openpyxl import load_workbook

wb = Workbook()
wb.save(path)


writer = pd.ExcelWriter(path, engine='openpyxl')
df3.to_excel(writer, sheet_name = 'Antenna IDs', index=False)
dfAntennaSettings.to_excel(writer, sheet_name = 'Antenna Settings', index=False)
df.to_excel(writer, sheet_name = 'Add powers to Prox', columns = ['Antenna','ID','EDT','Tech','Possible Ports','WhereToAddPower','Power (W)','Assess Freq'], index=False)
df.to_excel(writer, sheet_name = 'EMEG List', columns = ['ID','EDT','Sector','WhereToAddPower','Power (W)'], index=False)

#writer.close()

# load the Excel file with openpyxl
workbook = writer.book

for sheet in workbook:
    for column in range(1, sheet.max_column + 2):
        letter = get_column_letter(column)
        #sheet.column_dimensions[letter].auto_size = True
        sheet.column_dimensions[letter].bestFit = True


# save the modified workbook
workbook.save(path)



In [14]:
dfAntennaSettings.to_html(r'C:\Users\Mewtwo\Desktop\Antennadata.html', index=False)
df3.to_html(r'C:\Users\Mewtwo\Desktop\Antennadata2.html',index=False)
df.to_html(r'C:\Users\Mewtwo\Desktop\Antennadata3.html', columns = ['Antenna','ID','EDT','Tech','Possible Ports','WhereToAddPower','Power (W)','Assess Freq'], index=False)
df.to_html(r'C:\Users\Mewtwo\Desktop\Antennadata4.html',columns = ['ID','EDT','Sector','WhereToAddPower','Power (W)'],index=False)
import subprocess
subprocess.call('wkhtmltoimage -f png --width 0 Antennadata.html Antennadata.png', shell=True)


1

In [15]:

for sheetname in workbook.sheetnames:
    worksheet = workbook[sheetname]

    # set best fit attribute to true for all columns
    for column_cells in worksheet.columns:
        length = max(len(str(cell.value)) for cell in column_cells)
        worksheet.column_dimensions[column_cells[0].column_letter].width = length + 2
        worksheet.column_dimensions[column_cells[0].column_letter].bestFit = True

worksheet = workbook.active

# set the width of column A to 15
column_letter = get_column_letter(1)  # A
column_dimension = worksheet.column_dimensions[column_letter]
column_dimension.width = 15
column_dimension.bestFit = True


In [103]:
with open("C:\\Users\\Mewtwo\\Downloads\\RFNSA Scrapper_V2.html", encoding="utf8") as html_file:
    content = html_file.read()

# Get rid off prompts and source code
content = content.replace("div.input_area {","div.input_area {\n\tdisplay: none;")    
content = content.replace(".prompt {",".prompt {\n\tdisplay: none;")

f = open(FILE, 'w')
f.write(content)
f.close()

NameError: name 'FILE' is not defined

In [225]:
#jupyter nbconvert --to pdf 'RFNSA Scrapper_V2.ipynb' --template=hidecode.tplx


SyntaxError: invalid syntax (<ipython-input-225-51cd73a1ac0b>, line 1)